In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pyecharts import options as opts
from example.commons import Collector
from pyecharts.charts import Line


In [ ]:
# 回测参数
start_date =  '2018-01-01'
end_date = '2019-08-01'
freq = '2b'
# ML模型参数
max_round = 800
gpus = True
r = requests.post('http://10.63.6.110:8987/backtest', data={'start_date': start_date,
                                                            'end_date': end_date,
                                                            'freq': freq,
                                                            'max_round': max_round,
                                                            'GPUs': gpus})

In [ ]:
result = json.loads(r.text)

In [ ]:
ret_df = pd.DataFrame(json.loads(result['ret_df']))

In [ ]:
ret_df[['xgb_regress', 'net_xgb_regress']].cumsum().plot(figsize=(12, 6), 
                                                         title='Fixed freq rebalanced: {0}'.format('1b'))

In [ ]:
from pyecharts import options as opts
from example.commons import Collector
from pyecharts.charts import Line

plot = ret_df[['xgb_regress', 'net_xgb_regress']].cumsum()
v1 = list(plot.index)
v2 = list(plot.xgb_regress)
v3 = list(plot.net_xgb_regress)

line_chart = (
    Line()
    .add_xaxis(v1)
    .add_yaxis("xgb_regress", v2)
    .add_yaxis("net_xgb_regress", v3)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-1%")],
        title_opts=opts.TitleOpts(title='Fixed freq rebalanced: {0}'.format(freq)),
    )
)

line_chart.render_notebook()

In [ ]:
print("年化收益: {0:.2f}".format(len(ret_df) * np.mean(ret_df['net_xgb_regress'])))
print("夏普比率: {0:.2f}".format(np.sqrt(126) * np.mean(ret_df['net_xgb_regress']/ np.std(ret_df['net_xgb_regress']))))
